<a href="https://colab.research.google.com/github/plthiyagu/Personnel/blob/master/Password_encrypt_and_decrypt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Fernet

  Created wheel for Fernet: filename=fernet-1.0.1-cp38-none-any.whl size=2247 sha256=28251b1c5a5e1e9f28f196bd85eeaf18d1d782165bf4c8d84f361115d927b5a0
  Stored in directory: C:\Users\plthi\AppData\Local\pip\Cache\wheels\f3\43\c3\8ceadf7f614a43784db0955f4c8957af6a083a060c564b5242
  Created wheel for pyaes: filename=pyaes-1.6.1-cp38-none-any.whl size=26350 sha256=bd75c7bf03d33b58dfe88a1912f6ace3fa786d4f618e3a432ee0e87231fd1c01
  Stored in directory: C:\Users\plthi\AppData\Local\pip\Cache\wheels\bd\cf\7b\ced9e8f28c50ed666728e8ab178ffedeb9d06f6a10f85d6432
Successfully built Fernet pyaes


In [2]:
import secrets
from base64 import urlsafe_b64encode as b64e, urlsafe_b64decode as b64d

from cryptography.fernet import Fernet
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC

backend = default_backend()
iterations = 100_000

def _derive_key(password: bytes, salt: bytes, iterations: int = iterations) -> bytes:
    """Derive a secret key from a given password and salt"""
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(), length=32, salt=salt,
        iterations=iterations, backend=backend)
    return b64e(kdf.derive(password))

def password_encrypt(message: bytes, password: str, iterations: int = iterations) -> bytes:
    salt = secrets.token_bytes(16)
    key = _derive_key(password.encode(), salt, iterations)
    return b64e(
        b'%b%b%b' % (
            salt,
            iterations.to_bytes(4, 'big'),
            b64d(Fernet(key).encrypt(message)),
        )
    )

def password_decrypt(token: bytes, password: str) -> bytes:
    decoded = b64d(token)
    salt, iter, token = decoded[:16], decoded[16:20], b64e(decoded[20:])
    iterations = int.from_bytes(iter, 'big')
    key = _derive_key(password.encode(), salt, iterations)
    return Fernet(key).decrypt(token)


ModuleNotFoundError: No module named &#39;cryptography&#39;

In [6]:
message = 'John Doe'
password = 'mypass'
password_encrypt(message.encode(), password)
token = '9Ljs-w8IRM3XT1NDBbSBuQABhqCAAAAAAFyJdhiCPXms2vQHO7o81xZJn5r8_PAtro8Qpw48kdKrq4vt-551BCUbcErb_GyYRz8SVsu8hxTXvvKOn9QdewRGDfwx'
password_decrypt(token, password).decode()


'John Doe'